Covid-19 Dashboard, Jamie Phillips, 2020, 200873723, ec20933@qmul.ac.uk 

# __Covid-19 Dashboard__   

In [58]:
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [59]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

In [60]:
# Load JSON files and store the raw data in some variable. Edit as appropriate
jsondata={}

filters = [
    'areaType=nation',  #each metric-value pair is inside one string
    'areaName=Scotland'
]

structure = {
    "date": "date",
    "cases": "newCasesByPublishDate",
    "hospital": "newAdmissions",   
}

api = Cov19API(filters=filters, structure=structure)

timeseries=api.get_json()






In [61]:
import json
with open("timeseries.json", "wt") as OUTF:
    json.dump(timeseries, OUTF)

with open("timeseries.json", "rt") as INFILE:
    data=json.load(INFILE)  
    
datalist = data['data']



In [62]:
dates=[dictionary['date'] for dictionary in datalist ]
dates.sort()


In [63]:
def parse_date(datestring):
    """ Convert a date string into a pandas datetime object """
    return pd.to_datetime(datestring, format="%Y-%m-%d")

In [64]:
startdate=parse_date(dates[0])
enddate=parse_date(dates[-1])
print (startdate, ' to ', enddate)

2020-01-03 00:00:00  to  2020-11-26 00:00:00


In [65]:
index=pd.date_range(startdate, enddate, freq='D')
timeseriesdf=pd.DataFrame(index=index, columns=['cases', 'hospital'])


In [66]:
for entry in datalist: # each entry is a dictionary with date, cases, hospital
    date=parse_date(entry['date'])
    for column in ['cases', 'hospital']:
        # check that nothing is there yet - just in case some dates are duplicated,
        # maybe with data for different columns in each entry
        if pd.isna(timeseriesdf.loc[date, column]): 
            # replace None with 0 in our data 
            value= float(entry[column]) if entry[column]!=None else 0.0
            # this is the way you access a specific location in the dataframe - use .loc
            # and put index,column in a single set of [ ]
            timeseriesdf.loc[date, column]=value
            
# fill in any remaining "holes" due to missing dates
timeseriesdf.fillna(0.0, inplace=True)


In [67]:
timeseriesdf.to_pickle("timeseriesdf.pkl")

In [68]:
# our API access function. This will be called by the button when it is clicked
def access_api(button):
    # Ignore the parameter, put code for polling the API here
    print("I'm downloading data from the API...")
    print("...all done.")

# see the doc for the parameters    
apibutton=wdg.Button(
    description='Refresh data',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to download current Public Health England data',
    icon='download' # (FontAwesome names without the `fa-` prefix)
)

def refresh_graph():
    current=scale.value
    if current==scale.options[0]:
        other=scale.options[1]
    else:
        other=scale.options[0]
    scale.value=other # forces the redraw
    scale.value=current # now we can change it back

# register the callback function with the button
apibutton.on_click(access_api)


# this is an iPython function that generalises print for Jupyter Notebooks; we use it to 
# display the widgets


## Covid-19 graph for Scotland data

In [69]:
# in the dashboard, this should be computed from the json files
timeseriesdf=pd.read_pickle("timeseriesdf.pkl")


In [70]:

series=wdg.SelectMultiple(
    options=['cases', 'hospital',],
    value=['cases', 'hospital',],
    rows=3,
    description='Stats:',
    disabled=False
)

scale=wdg.RadioButtons(
    options=['linear', 'log'],
#    value='pineapple', # Defaults to 'pineapple'
#    layout={'width': 'max-content'}, # If the items' names are long
    description='Scale:',
    disabled=False
)

# try replacing HBox with a VBox
controls=wdg.HBox([series, scale])

def timeseries_graph(gcols, gscale):
    if gscale=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0:
        timeseriesdf[list(gcols)].plot(logy=logscale)
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")

# keep calling timeseries_graph(gcols=value_of_series, gscale=value_of_scale); capture output in variable graph   
graph=wdg.interactive_output(timeseries_graph, {'gcols': series, 'gscale': scale})



In [71]:
# stack series and scale on top of each other
ctrls=wdg.VBox([series, scale])
# put the graph and the controls side by side
form=wdg.HBox([graph, ctrls])

# Now form contains the entire interface
display(form)
display(apibutton)

Button(description='Refresh data', icon='download', style=ButtonStyle(), tooltip='Click to download current Pu…

I'm downloading data from the API...
...all done.


___ Click the button above to refresh the data ___

*Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).*